# Project 1: Image Classification

## Task 0: Getting Started

Read the getting started guide titled **"Python for Deep Learning"** and get familiar with Python and PyTorch. Read the provided code below and get familiar with the commands and their parameters to understand what the code is trying to do. We recommend to spend a fair amount of time to understand all the different parts of the code. This understanding will be important for this and future projects.

The goal of this project is to implement the *“Hello World!”* program of deep learning: designing and training a network that performs image classification. The dataset we will be using is CIFAR10 which is a large set of images that are classified into 10 classes (airplane, bird, cat, etc.).

## Task 1:  Data Loading (10 points)
Complete the **DataLoader** below which we will use to load images of the cifar10 dataset provided by torchvision. Your task is to normalize it by shifting and scaling it by a factor of 0.5. For the training set, introduce random transformations (e.g. flips) for data augmentation.

In [ ]:
from __future__ import print_function, division

import os
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
print(torch.__version__)
from PIL import Image
import matplotlib.pyplot as plt
plt.ion()   # interactive mode

# Data augmentation, tensor conversion and normalization for training
# Just normalization and tensor conversion for testing
data_transforms = {
    'train': transforms.Compose([
      # TODO: Task 1
    ]),
    'test': transforms.Compose([
     # TODO: Task 1
    ])
}

# Download (if not downloaded before) & Load CIFAR10
torch.random.manual_seed(0)
image_datasets = {x: torchvision.datasets.CIFAR10(root='./data', train=(x=='train'), download=True, transform=data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=(x=='train'), num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

# Ship to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"The classes found in CIFAR-10 are: {class_names}")

### Visualize a few images

Let’s visualize a few training images so as to understand the data augmentations. The results should look like:

<img src="https://i.imgur.com/Sa6l1go.png" width="400" align="left">

In [ ]:
# TODO Task 1:  Run this cell and try to understand the output of each step
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## Task 2: Basic Networks (20 points)
1. Create a Fully connected Network (FcNet) using the following layers in the Jupyter Notebook (use the non-linearities wherever necessary):
```
FcNet(
  (fc1): Linear(in_features=3072, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=84, bias=True)
  (fc4): Linear(in_features=84, out_features=10, bias=True)
)
```
Train the FcNet for **3** epoches and record the training time and accuracy in your final report.

2. Create a Convolutional Network (ConvNet) using the following layers in the Jupyter Notebook (use the non-linearities wherever necessary):
```
ConvNet(
  (conv): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
```
Train the ConvNet for **3** epoches and record the training time and accuracy in your final report. Note: You can reuse the conv layers to match the in_features of fc1. 

*Use the default SGD optimizer ( lr=0.001, momentum=0.9) for training.

### Model training code 

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25, save_path='saved_weight.pth'):
    since = time.time()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train': model.train()  # Set model to training mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    torch.save(model.state_dict(), save_path)
    return model

### Model test code 

In [ ]:
def test_model(model, load_path='saved_weight.pth'):    
    # load the model weights
    model.load_state_dict(torch.load(load_path))
    
    since = time.time()

    for phase in ['test']:
        if phase == 'test':
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
           

            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

            # statistics
            running_corrects += torch.sum(preds == labels.data)
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Acc: {:.4f}'.format(phase, epoch_acc))

    time_elapsed = time.time() - since
    print('Testing complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return 

### 1) FC Network

In [ ]:
# 1) Define a Fully Connected Neural Network
# Please advise the Pytorch Documentation and use the appropiate calls

class FcNet(nn.Module):
    def __init__(self):
        super(FcNet, self).__init__()
        # TODO Task 2:  Define the layers 

    def forward(self, x):
        # TODO Task 2:  Define the forward pass
        
        return x

model_ft = FcNet() #Define the model
model_ft = model_ft.to(device) 
print(model_ft)

# TODO Task 2:  Define loss criterion - cross entropy loss
# TODO Task 2:  Define Optimizer
# TODO Task 2:  Train the model
# TODO Task 2:  Test the model

### 2) CNN

In [ ]:
# 2) Define a Convolutional Neural Network
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # TODO Task 2:  Define the CNN layers 

    def forward(self, x):
        # TODO Task 2:  Define the forward pass
        
        return x

model_ft = ConvNet() #Define the model
model_ft = model_ft.to(device)
print(model_ft)
# TODO Task 2:  Define loss criterion - cross entropy loss
# TODO Task 2:  Define Optimizer
# TODO Task 2:  Train the model
# TODO Task 2:  Test the model

## Task 3.A: Design Your Network (30 points)
Define your own Convolutional Network (MyNet) based on the [ResNet paper](https://arxiv.org/pdf/1512.03385.pdf) architecture (see Sec. 4.2 for the original architecture used). Here we will experiment with different configurations. Add following modifications and train the Network for **25** epoches. Keep the best settings for each step (for each step, record the training accuracy of the last epoch and test accuracy in your report):

- Modify the number of ResNet blocks per layer: For Simplicity, implement a three-layered ResNet architecture. For each layer, try 1 , 2 and 3 number of ResNet blocks (3 configurations in total). You can choose any of the downsampling methods to match the tensor sizes wherever necessary. Use 2D average pooling layer before applying the final linear layer. For the three layers keep the number of filters 16, 32 and 64 respectively. Follow the ResNet paper to select the kernel size, paddings, optimizer/learning rate, strides, activations and **Batch Normalization** (select a suitable batch size) layers for this task. Print the model summary of the selected model.

#### *Bonus Points: Define a validation set within the training set to monitor underfitting/overfitting after every epoch for each task. (Hint modify dataloaders and/or train_model function) 

### Design Your Network

In [ ]:
# Task 3.A: Configuration 1

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        # TODO Task 3: Design Your Network

    def forward(self, x):
        # TODO Task 3: Design Your Network
        return x

In [ ]:
# Task 3.A: Configuration 2


In [ ]:
# Task 3.A: Configuration 3


## Task 3.B: Design Your Network (10 points)
Using your best network/model from Task 3.A, please do the following modifications:

- Use Dropout: Toggle **Dropout** in fully connected layers. Does it improve the validation/test accuracy and/or avoid overfitting?

In [ ]:
# Task 3.B: Using Dropout

## Task 4: The Optimizer (20 points)
Keeping the best settings of Task 4, try 2 different optimizers (SGD and ADAM) with 3 different learning rates (0.01, 0.1, 1.) . Keep the other parameters as default. Plot the loss curves ( Training step vs Training loss ) for the 6 cases (Hint: Modify the train_model fuction). How does the learning rate affect the training?

#### *Bonus Points: Define a validation set within the training set to examine the best model among the above cases. (Hint modify dataloaders and/or train_model function) 

In [ ]:
# Define a Convolutional Neural Network
class MyFinalNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        # TODO Task 3: Design Your Network

    def forward(self, x):
        # TODO Task 3: Design Your Network
        return x

In [ ]:
model_ft = MyNet()
model_ft = model_ft.to(device)
# TODO:  Define loss criterion - cross entropy loss
# TODO Task 4: The Optimizer
## Train and evaluate

### Display model predictions

In [ ]:
## Display model predictions
## Generic function to display predictions for a few images

def display_predictions(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
# TODO Diplay your best model predictions

## Task 5: Feature Visualization (5 points)
Visualize feature maps of the first and the last convolutional layer of your final network using **cifar_example.jpg** as input image. Show the visualization in the report.

#### First layer activations
<img src="https://i.imgur.com/kGB9AuP.png" width="400" align="left">

#### Last layer activations

<img src="https://i.imgur.com/qelH05X.png" width="400" align="left">

In [ ]:
#Task 5: Visualization

In [ ]:
def transfer_single_img_to_tensor(img_path):
    im = Image.open(img_path)
    img = np.asarray(im)/255
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    
    inp = (img - mean) / std
    inp = np.asarray(inp, dtype=np.float32)
    inp = inp.transpose((2, 0, 1))
    inp = np.expand_dims(inp, axis=0)
    inp = torch.from_numpy(inp, )
    inputs = inp.to(device)
    return inputs

In [ ]:
def feature_imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.detach().numpy().transpose((1, 2, 0))
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
inputs = transfer_single_img_to_tensor('example_imgs/cifar_example.jpg') # Loads an image and normalizes it
model_ft.eval()
with torch.no_grad():
   # TODO: Retrive the first and the last layer feature maps of your best model. (Hint: Move back to CPU)

In [ ]:
# Visualize the feature maps
out = torchvision.utils.make_grid(# TODO: feaure map 1)
feature_imshow(out)

In [ ]:
out = torchvision.utils.make_grid(# TODO: feaure map 2)
feature_imshow(out)

## Task 6: Weight Visualization (5 points)
Visualize weights of convolutional layers of your final network. Show the visualization in the report.

In [ ]:
# TODO: Task 6
# Hint: 
# 1) What is the size of each weight (filter) tensor?
# 2) Normalize weight values to [0, 1]
# 3) Each subfigure should be of size [kernel_size, kernel_size]
# 4) How many subfigures in total?